In [2]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [3]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [4]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1542659, 2)

In [5]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()



,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,MetLife
1,a1W8c000007vDN9EAM,Metlife Holding Metlife Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,MetLife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,MetLife
3,a1W8c000007vDNBEA2,Metlife Holding Metlife Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,MetLife
4,a1W8c000007vDNCEA2,Metlife Holding Metlife Carlos Escarate Y Cia ...,A-0004,0018c00002axQXvAAM,MetLife


In [6]:
Empresas_SF.shape

(7925, 5)

In [7]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [8]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [9]:
pacientes_y_union.shape

(1650110, 4)

In [10]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Metlife Vulco Vulco S.A.,A-1075,0018c00002cXJZeAAO,MetLife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083laZEAQ,a1W8c0000083laZEAQ,EY - EY Audit SpA,A-4489,0018c00002eTboVAAS,EY
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [11]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,Codigo Ficha,rut1,Empleador,CreatedDate,Canal,Cod. Empresa Emp_pro,Id Paciente,Tipo Ficha,clínica,Mes,Año,Dia,Fecha,Fecha3,rut
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 18:31:33,24-01-2023,181523778
1,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 19:19:01,24-01-2023,184478412
2,FFF-0038,150356237,Asociación Chilena De Seguridad,2023-01-24T19:21:28.000+0000,Saliente - Llamada,a1Y8c00000CkvAuEAJ,0018c00002axzuCAAQ,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 19:21:28,24-01-2023,150356237
3,FFF-0036,154370366,Asociación Chilena De Seguridad,2023-01-24T18:41:51.000+0000,Saliente - Llamada,a1Y8c00000CkvYPEAZ,0018c00002ay05tAAA,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 18:41:51,24-01-2023,154370366
4,FFF-0039,197818735,NaN,2023-01-24T20:27:11.000+0000,Entrante - WhatsApp,NaN,0018c00002eit30AAA,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 20:27:11,24-01-2023,197818735


In [12]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

KeyError: 'nombres'

In [13]:
Pacientes_Archivo.head()

,Codigo Ficha,rut1,Empleador,CreatedDate,Canal,Cod. Empresa Emp_pro,Id Paciente,Tipo Ficha,clínica,Mes,Año,Dia,Fecha,Fecha3,rut
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 18:31:33,24-01-2023,181523778
1,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 19:19:01,24-01-2023,184478412
2,FFF-0038,150356237,Asociación Chilena De Seguridad,2023-01-24T19:21:28.000+0000,Saliente - Llamada,a1Y8c00000CkvAuEAJ,0018c00002axzuCAAQ,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 19:21:28,24-01-2023,150356237
3,FFF-0036,154370366,Asociación Chilena De Seguridad,2023-01-24T18:41:51.000+0000,Saliente - Llamada,a1Y8c00000CkvYPEAZ,0018c00002ay05tAAA,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 18:41:51,24-01-2023,154370366
4,FFF-0039,197818735,NaN,2023-01-24T20:27:11.000+0000,Entrante - WhatsApp,NaN,0018c00002eit30AAA,0128c000002CYubAAG,Orientación Médica,1,2023,24,2023-01-24 20:27:11,24-01-2023,197818735


In [14]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,Codigo Ficha,rut1,Empleador,CreatedDate,Canal,Cod. Empresa Emp_pro,Id Paciente,Tipo Ficha,clínica,Mes,...,rut,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,0128c000002CYubAAG,Orientación Médica,1,...,181523778,0018c00002amVPcAAM,181523778,0018c00002amVPcAAM,a1W8c0000083j8vEAA,a1W8c0000083j8vEAA,Farmacias Ahumada S.A.,A-2321,0018c00002cXclUAAS,Farmacias Ahumada
1,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,0128c000002CYubAAG,Orientación Médica,1,...,184478412,0018c00002ay08PAAQ,184478412,0018c00002ay08PAAQ,a1W8c000007vDSoEAM,a1W8c000007vDSoEAM,Asociación Chilena De Seguridad,A-0352,0018c00002axQdUAAU,MetLife
2,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,0128c000002CYubAAG,Orientación Médica,1,...,184478412,0018c00002ay08PAAQ,184478412,0018c00002ay08PAAQ,a1W8c000007vDQvEAM,a1W8c000007vDQvEAM,Metlife Holding Metlife Fdo. De Salud Naci De ...,A-0235,0018c00002axQbeAAE,MetLife
3,FFF-0038,150356237,Asociación Chilena De Seguridad,2023-01-24T19:21:28.000+0000,Saliente - Llamada,a1Y8c00000CkvAuEAJ,0018c00002axzuCAAQ,0128c000002CYubAAG,Orientación Médica,1,...,150356237,0018c00002axzuCAAQ,150356237,0018c00002axzuCAAQ,a1W8c000007vDSoEAM,a1W8c000007vDSoEAM,Asociación Chilena De Seguridad,A-0352,0018c00002axQdUAAU,MetLife
4,FFF-0036,154370366,Asociación Chilena De Seguridad,2023-01-24T18:41:51.000+0000,Saliente - Llamada,a1Y8c00000CkvYPEAZ,0018c00002ay05tAAA,0128c000002CYubAAG,Orientación Médica,1,...,154370366,0018c00002ay05tAAA,154370366,0018c00002ay05tAAA,a1W8c000007vDSoEAM,a1W8c000007vDSoEAM,Asociación Chilena De Seguridad,A-0352,0018c00002axQdUAAU,MetLife


In [15]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/reportes/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [16]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel(
     "F:/Empresas SF/reportes/account.xlsx", index=False
)
Subir_pacientes.head()


KeyError: 'RutEstaoNo'